In [2]:
import json
import pymysql

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import re

import pandas as pd

In [3]:
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(options=options)

In [4]:
driver.get("https://chat.openai.com/chat")

In [10]:
queryBar = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div[2]/form/div/div[2]/textarea')
queryBar.send_keys("GPT 너에 대해서 간략히 소개해줄래?")
time.sleep(2)
queryBar.send_keys("\n")

In [ ]:
target = '//*[@id="__next"]/div[1]/div[1]/main/div[2]/form/div/div[2]/button/svg'
element = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located((By.XPATH, target))
)

answer = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div[1]/div/div/div/div[2]/div/div[2]')
print(answer.text)